<a href="https://colab.research.google.com/github/pks98/NLP_Reviews/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [ ]:
# pip install tensorflow-text

In [3]:
import pandas as pd

df = pd.read_csv("final_reviews.csv")
df.head()

,Review_Title,Reviews,Rating
0,AWESOME...IN LOVE WITH IT !,Really impressed with the fastest delivery of ...,5
1,Enjoy while it lasts,This is the second time I am writing this revi...,2
2,Good,1. Quite good look2. Better noise cancellation...,4
3,not great,its a good set of ear buds but1. the quality o...,1
4,Best buds in the budget with Type C,I have been researching about getting a nice T...,5


In [4]:
def data_clean(text):
  if text=='The media could not be loaded.':
    return " "
  else:
    return text
df['Reviews']=df['Reviews'].apply(data_clean)
df.fillna('',inplace=True)
df['text']=df['Review_Title']+' '+df['Reviews']
df.sample(10)

,Review_Title,Reviews,Rating,text
4384,Good Product but will not fit in small ears,do not buy if:1. your ear is small2. You liste...,4,Good Product but will not fit in small ears do...
1354,No noise cancellation,,4,No noise cancellation
1259,Everything is defective. This shouldn't be all...,This product shouldn't be allowed to put on sa...,1,Everything is defective. This shouldn't be all...
3795,Good Soung Great Price,This is pretty much a good pair of wireless ea...,4,Good Soung Great Price This is pretty much a g...
2276,Poor bluetooth connection quality (revised rev...,"Not good for calls , only for music.During cal...",1,Poor bluetooth connection quality (revised rev...
3503,Non durable,They worked really well for the duration they ...,2,Non durable They worked really well for the du...
202,Amazing sound quality and battery life,"It has very good battery backup, The sound qua...",5,Amazing sound quality and battery life It has ...
2944,The earbuds on the right is not even working 😡...,,4,The earbuds on the right is not even working 😡...
2322,Great earphones,Other than the near to no noise cancelling fea...,5,Great earphones Other than the near to no nois...
800,Not compatible for calls on laptop (MS teams /...,"The product is not compatible with MS teams, o...",1,Not compatible for calls on laptop (MS teams /...


In [5]:
df=df[df['Rating']!=3.0]

def rating_classify(rating):
  if rating>=4:
    return 1
  else:
    return 0
    
df['label']=df['Rating'].apply(rating_classify)
df.sample(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Review_Title,Reviews,Rating,text,label
4253,One of the earbuds stopped working,There was an issue since first day but it used...,1,One of the earbuds stopped working There was a...,0
1828,Best budget tws,"The sound quality is good ,,its more towards t...",5,"Best budget tws The sound quality is good ,,it...",1
3883,"Faulty, will not work for long periods of time...",The earbuds I had stopped working properly aft...,2,"Faulty, will not work for long periods of time...",0
2048,Worth buying,It's been a month i am using this airdrops and...,5,Worth buying It's been a month i am using this...,1
1354,No noise cancellation,,4,No noise cancellation,1


In [6]:
df.drop(['Review_Title','Reviews','Rating'],axis=1,inplace=True)
df.head()

,text,label
0,AWESOME...IN LOVE WITH IT ! Really impressed w...,1
1,Enjoy while it lasts This is the second time I...,0
2,Good 1. Quite good look2. Better noise cancell...,1
3,not great its a good set of ear buds but1. the...,0
4,Best buds in the budget with Type C I have bee...,1


In [7]:
df.groupby('label').describe()

text                           
      count unique            top freq
label                                 
0      1691   1690    Don't buy      2
1      2296   2291  Good product     4

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['text'],df['label'], stratify=df['label'],random_state=5)

In [10]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [18]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.2, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [16]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

In [13]:
# METRICS = [
#       tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#       tf.keras.metrics.Precision(name='precision'),
#       tf.keras.metrics.Recall(name='recall')
# ]

# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=METRICS)

In [19]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30)

Epoch 1/30
94/94 [==============================] - 104s 1s/step - loss: 0.7023 - accuracy: 0.5592 - val_loss: 0.6411 - val_accuracy: 0.5928
Epoch 2/30
94/94 [==============================] - 100s 1s/step - loss: 0.6528 - accuracy: 0.6140 - val_loss: 0.6140 - val_accuracy: 0.7482
Epoch 3/30
94/94 [==============================] - 100s 1s/step - loss: 0.6221 - accuracy: 0.6659 - val_loss: 0.5906 - val_accuracy: 0.6871
Epoch 4/30
94/94 [==============================] - 100s 1s/step - loss: 0.5961 - accuracy: 0.6893 - val_loss: 0.5719 - val_accuracy: 0.7733
Epoch 5/30
94/94 [==============================] - 100s 1s/step - loss: 0.5922 - accuracy: 0.6920 - val_loss: 0.5594 - val_accuracy: 0.6941
Epoch 6/30
94/94 [==============================] - 100s 1s/step - loss: 0.5751 - accuracy: 0.7050 - val_loss: 0.5491 - val_accuracy: 0.7091
Epoch 7/30
94/94 [==============================] - 100s 1s/step - loss: 0.5658 - accuracy: 0.7227 - val_loss: 0.5405 - val_accuracy: 0.7874
Epoch 8/30
94